In [25]:
!pip install ipython-autotime
%load_ext autotime
!pip install pyod
!pip install spotipy

import pandas as pd
import numpy as np
from tqdm import tqdm # tqdm mostra barra de processo https://github.com/tqdm/tqdm

import os
import glob

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 11.2 s (started: 2022-06-05 14:00:13 +00:00)


___
## Criando dataset global

In [ ]:
# Usuário: fijita8647@karavic.com
# Senha: INF10322803
cid ="" 
secret = ""

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

time: 3.95 ms (started: 2022-06-05 14:00:25 +00:00)


O path abaixo é para o Top 200 global

A funcao abaixo monta um dataframe com as URIs de todas as musicas que entraram em um dado Top 200

In [27]:
def pega_uri_charts(path):

  #Pega todos os arquivos do diretorio
  arquivos = glob.glob(os.path.join(path,"*.csv"))
  dados = pd.DataFrame()

  for file in arquivos:
      dados_temp = pd.read_csv(file)
      #Coloca o nome certo nas colunas
      dados_temp.columns = dados_temp.iloc[0]
      dados_temp = dados_temp.drop(axis=0,index=0)
      #Vai concatenando no dataet final
      dados = pd.concat([dados,dados_temp])

  #Preciso apenas do URI, Streams, Nome e Artista
  dados = dados[["URL","Streams","Track Name","Artist"]]

  #Para analise de anomalias nao preciso da informacao de quantas vezes a musica apareceu no top 200, entao removo duplicatas
  dados = dados.drop_duplicates(keep="first")

  #Reset no index, ja que removi linhas
  dados = dados.reset_index(drop=True)
  
  return dados




time: 14.4 ms (started: 2022-06-05 14:00:25 +00:00)


A função abaixo pega as features de um track via URI do track

In [49]:
def get_features_from_tracks(track_id):
  #Pega as features da musica e outras informações importantes e/ou interessantes
  
  #Audio features
  features = sp.audio_features(track_id)

  #Informacoes sobre a musica
  # data_lancamento = sp.track(track_id)['album']['release_date']
  # nome = sp.track(track_id)['name']
  # popularidade_musica = sp.track(track_id)['popularity']
  
  #Informacoes sobre o artista
  # nome_artista = sp.track(track_id)['artists'][0]['name']
  # uri_artista = sp.track(track_id)["artists"][0]['uri']
  # popularidade_artista = sp.artist(uri_artista)['popularity']
  # estilo = sp.artist(uri_artista)['genres']

  
  #Montando o dataframe
  features = pd.DataFrame(features)

  # features["nome"] = nome
  # features["data_lancamento"] = data_lancamento
  # features["data_lancamento"] = pd.to_datetime(features["data_lancamento"])
  # features["mes_lancamento"] = features["data_lancamento"].dt.month
  # features["dia_lancamento"] = features["data_lancamento"].dt.day
  # features["dia_semana_lancamento"] = features["data_lancamento"].dt.day_name()
  # print(type(nome_artista))
  # features["artista"] = nome_artista
  # features["popularidade_musica"] = popularidade_musica
  # features["popularidade_artista"] = popularidade_artista

  # if(estilo != []):
  # 	  features["estilo_musical"] = estilo[-1] #sempre é uma lista e eu, arbritariamente, escolhi pegar o ultimo estilo da lista
  # else:
  #   features["estilo_musical"] = "indefinido"
  
  
  return features


time: 19.1 ms (started: 2022-06-05 16:27:58 +00:00)


A função abaixo pega as audio features de cada um dos tracks e monta um dataframe

In [61]:
def gera_dataset(lista_uris_tracks):
  df_features_final = pd.DataFrame()
  tam = lista_uris_tracks.shape[0]
  print(tam)
  i = 0
  for idx,track_uri in lista_uris_tracks.iterrows():
    
    #Eu pego as features da musica
    df_features_parcial = get_features_from_tracks(track_uri.URL)

    df_features_parcial["streams"] = track_uri.Streams
    df_features_parcial["nome"] = track_uri["Track Name"]
    df_features_parcial["artista"] = track_uri.Artist
    # E concateno no df final
    df_features_final = pd.concat([df_features_final,df_features_parcial],axis=0)
    i+=1
    perc = 100*i/tam
    if(perc%1==0):
      print("%.2f"%perc)
  
  if("Unnamed: 0" in df_features_final.columns):
    df_features_final = df_features_final.drop(axis=1,columns=["Unnamed: 0"])
  

  return df_features_final

time: 29.6 ms (started: 2022-06-05 16:35:11 +00:00)


In [30]:
path_global = "/content/drive/MyDrive/INF1032 - Spotify/Dados/Anna/csv/global"

time: 806 µs (started: 2022-06-05 14:00:25 +00:00)


In [ ]:
path_consolidados = '/content/drive/My Drive/INF1032 - Spotify/Dados/ChartsAPI/'

In [62]:
lista_uris_global = pega_uri_charts(path_global)
df_musicas_global = gera_dataset(lista_uris_tracks=lista_uris_global)
df_musicas_global.to_csv(path_consolidados+"features_musica_top200_global_consolidado.csv") 

55800


KeyboardInterrupt: ignored

time: 21.6 s (started: 2022-06-05 16:35:17 +00:00)


In [56]:
gera_dataset(lista_uris_global.iloc[0:1])

1
0.00


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,streams,nome,artista
0,0.52,0.731,6,-5.338,0,0.0557,0.342,0.00101,0.311,0.662,...,audio_features,4LRPiXqCikLlN15c3yImP7,spotify:track:4LRPiXqCikLlN15c3yImP7,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,https://api.spotify.com/v1/audio-analysis/4LRP...,167303,4,65611902,As It Was,Harry Styles


time: 91 ms (started: 2022-06-05 16:31:22 +00:00)


Pasta onde esses dataframes serão salvos

In [ ]:
# df_musicas_global = gera_dataset(lista_uris_tracks=lista_uris_global.iloc[:])
# df_musicas_global.to_csv(path_consolidados+"features_artista_top200_global_consolidado.csv") 

Gerando os dataframes para as demais pastas de csvs.

Apenas audio features

In [ ]:
paths = ["/content/drive/MyDrive/INF1032 - Spotify/Dados/Anna/csv/br",
         "/content/drive/MyDrive/INF1032 - Spotify/Dados/Anna/csv/de",
         "/content/drive/MyDrive/INF1032 - Spotify/Dados/Anna/csv/in",
         "/content/drive/MyDrive/INF1032 - Spotify/Dados/Anna/csv/us",
         "/content/drive/MyDrive/INF1032 - Spotify/Dados/Anna/csv/mx"]

for path in paths:
  pais = path[-2:]
  lista_uris = pega_uri_charts(path_consolidados)
  df_musicas_consolidadas = gera_dataset(lista_uris_tracks=lista_uris)
  df_musicas_consolidadas.to_csv(path_consolidados + "features_musica_top200_" + pais + "_consolidado.csv")
  print(pais,"Salvo") 

Apenas features do artista

In [ ]:
for path in paths:
  pais = path[-2:]
  # lista_uris = pega_uri_charts(path_consolidados)
  # df_musicas_consolidadas = gera_dataset(lista_uris_tracks=lista_uris)
  # df_musicas_consolidadas.to_csv(path_consolidados + "features_artista_top200" + pais + "_consolidado.csv")
  # print(pais,"Salvo") 